In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS

Pour les plages de dates disponibles :
* NEET et variables explicatives : $2013-2018$
* Variables economiques : $2000-2019$
* Variables d'éducation : $2013-2017$

Pour le NEET rate, j'ai pris les $15-29$ ans.

Pour les variables qui dépendent du niveau d'éducation, j'ai pris $L1$ partout.

# Import data and create panel data sets

## Import data

In [2]:
panel_data_l1 = pd.read_csv('./data/panel_data/panel_data_l1.csv')
df_eco_features = pd.read_csv('./data/panel_data/economic_features.csv')
df_educ_features_l1 = pd.read_csv('./data/panel_data/educ_features_l1.csv')
df_labour_features = pd.read_csv('./data/panel_data/labour_market.csv')

## Create panel data set for the NEET rate and the 3 explanatory variables

In [3]:
oecd_countries = {'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'CAN': 'Canada', 'CHL': 'Chile', 'COL': 'Colombia', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece', 'HUN': 'Hungary', 'ISL': 'Iceland', 'IRL': 'Ireland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 'KOR': 'Korea', 'LVA': 'Latvia', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MEX': 'Mexico', 'NLD': 'Netherlands', 'NZL': 'New Zealand', 'NOR': 'Norway', 'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden', 'CHE': 'Switzerland', 'TUR': 'Turkey', 'GBR': 'United Kingdom', 'USA': 'United States'}
code_countries = [code for code in oecd_countries.keys()]
years = [year for year in range(2013,2019)]

In [4]:
np.unique(panel_data_l1.Country)

array(['AUS', 'BEL', 'CAN', 'CHL', 'CZE', 'DEU', 'ESP', 'EST', 'FRA',
       'HUN', 'IRL', 'ISR', 'JPN', 'LTU', 'LUX', 'LVA', 'MEX', 'NLD',
       'NZL', 'POL', 'PRT', 'SVK', 'SVN', 'USA'], dtype=object)

In [5]:
bin_df = pd.DataFrame()
# create binary variables for each oecd country
for code in np.unique(panel_data_l1.Country):
    bin_var = (panel_data_l1.Country == code).astype(int)
    bin_df['bin_'+code] = bin_var

In [6]:
# create a group id for each different country for the clustered standard errors
groupid = []
previous_country,gid = 'AUS', 0
for code in panel_data_l1.Country:
    current_country = code
    if current_country==previous_country:
        groupid.append(gid)
    else:
        gid += 1
        groupid.append(gid)
    previous_country = current_country
panel_data_l1['groupid'] = groupid

In [7]:
panel_data_l1

,Country,Time,NEET,Exp_LMP,STR,Min_Wage,groupid
0,AUS,2013,13.015899,0.87,15.615,23283.766881,0
1,AUS,2014,12.647472,0.93,15.612,23356.492667,0
2,AUS,2015,11.831610,0.91,15.433,23641.395398,0
3,AUS,2016,11.352150,0.86,15.168,23915.419580,0
4,AUS,2017,10.946128,0.85,15.124,24128.731046,0
...,...,...,...,...,...,...,...
123,USA,2014,15.047262,0.28,15.435,16285.276127,23
124,USA,2015,14.380193,0.28,15.354,16265.980260,23
125,USA,2016,14.118049,0.26,15.216,16063.328012,23
126,USA,2017,13.280724,0.24,15.182,15728.297963,23


## Create panel data set for the economic features

In [8]:
df_eco_features

,Country,Time,GDP,CPI,DEBT
0,AUS,2000,21679.247842,4.457435,41.14750
1,AUS,2001,19490.861110,4.407135,40.40488
2,AUS,2002,20082.483267,2.981575,38.67284
3,AUS,2003,23447.031001,2.732596,35.66726
4,AUS,2004,30430.676437,2.343255,32.31054
...,...,...,...,...,...
652,USA,2015,56839.381774,0.118627,136.43000
653,USA,2016,57951.584082,1.261583,138.11100
654,USA,2017,60062.222313,2.130110,134.67420
655,USA,2018,62996.471285,2.442583,136.17960


In [9]:
gdp, debt = df_eco_features.GDP, df_eco_features.DEBT

In [10]:
lgdp, ldebt = np.log(gdp), np.log(debt)

In [11]:
df_eco_features.GDP, df_eco_features.DEBT = lgdp, ldebt
df_eco_features.columns = ['Country', 'Time', 'LogGDP', 'CPI', 'LogDEBT']

In [12]:
df_eco_features

,Country,Time,LogGDP,CPI,LogDEBT
0,AUS,2000,9.984111,4.457435,3.717163
1,AUS,2001,9.877701,4.407135,3.698951
2,AUS,2002,9.907603,2.981575,3.655138
3,AUS,2003,10.062499,2.732596,3.574233
4,AUS,2004,10.323206,2.343255,3.475393
...,...,...,...,...,...
652,USA,2015,10.947985,0.118627,4.915812
653,USA,2016,10.967363,1.261583,4.928058
654,USA,2017,11.003136,2.130110,4.902859
655,USA,2018,11.050834,2.442583,4.913975


## Create panel data set for the education indicators

In [13]:
df_educ_features_l1

,Country,Time,Years_schooling,Avg_class_size,Exp_educ
0,AUS,2013,12.6,23.725,9241.9922
1,AUS,2014,12.7,23.859,9257.9980
2,AUS,2015,12.8,23.821,9524.7178
3,AUS,2016,12.9,23.669,10022.5670
4,AUS,2017,12.9,23.613,10238.4130
...,...,...,...,...,...
128,GBR,2013,12.6,25.404,10615.3770
129,GBR,2014,12.7,25.296,11276.6310
130,GBR,2015,12.8,25.988,11715.1060
131,GBR,2016,12.9,25.937,11350.0200


## Create panel data set for the labour market indicators

In [14]:
df_labour_features

,Country,Time,protection_of_workers,short_time_workers,involuntary_pt_workers,ft_and_pt_employ,marginally_attached_workers,employees_bargain,years_schooling,avg_class_size,educ_spendings
0,AUS,2016,1.700,0.892538,27.952453,19.342875,5.644458,60.0,12.9,22.158,8795.3633
1,AUS,2014,1.700,0.532556,28.277395,18.917433,5.584873,60.1,12.7,23.785,8107.4346
2,AUT,2013,1.800,0.479700,11.341942,18.913490,3.342615,98.0,11.9,21.022,10486.6760
3,AUT,2014,1.800,0.392421,10.955043,19.574962,3.614805,98.0,12.1,20.977,10661.8130
4,AUT,2015,1.800,0.454537,11.852590,19.793941,3.678977,98.0,12.1,20.921,11193.4690
5,EST,2015,1.934,0.526973,12.138937,6.022114,4.059337,18.6,12.7,18.148,5838.8569
6,FIN,2014,2.518,0.466532,23.873874,7.418025,5.135235,89.3,12.4,19.656,8765.7246
7,FIN,2015,2.518,0.435119,25.849081,7.945387,5.302583,89.3,12.4,19.656,9286.6465
8,FRA,2014,2.812,0.077543,38.833678,12.448917,1.401585,98.5,11.4,25.262,6860.5566
9,DEU,2013,2.332,0.101407,14.437775,20.824191,1.469701,57.6,14.0,24.285,7958.0161


## Check for multicollinearity

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

# OLS on panel data

## Panel OLS

Je pense qu'il faut supprimer la variable `Min_Wage`, c'est elle qui pose des problèmes de multicollinéarité, et en plus ça facilite notre étude comme ça on a plus que 2 variables explicatives.

### Without state or time fixed effects

In [16]:
panel_data_l1_ols = panel_data_l1.set_index(['Country', 'Time'])
panel_data_l1_ols = sm.add_constant(panel_data_l1_ols)

In [80]:
basic_ols = PanelOLS(panel_data_l1_ols.NEET, panel_data_l1_ols[['const', 'Exp_LMP', 'STR']]).fit()
print(basic_ols)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   NEET   R-squared:                        0.1582
Estimator:                   PanelOLS   R-squared (Between):              0.1462
No. Observations:                 128   R-squared (Within):               0.1655
Date:                Tue, Mar 30 2021   R-squared (Overall):              0.1582
Time:                        09:51:28   Log-likelihood                   -341.29
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.744
Entities:                          24   P-value                           0.0000
Avg Obs:                       5.3333   Distribution:                   F(2,125)
Min Obs:                       2.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             11.744
                            

### With only state fixed effects

In [18]:
panel_ols_sfe = PanelOLS(panel_data_l1_ols.NEET, panel_data_l1_ols[['const', 'Exp_LMP', 'STR']], entity_effects=True).fit(cov_type='clustered', cluster_entity=True)

In [19]:
print(panel_ols_sfe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   NEET   R-squared:                        0.4193
Estimator:                   PanelOLS   R-squared (Between):             -0.7526
No. Observations:                 128   R-squared (Within):               0.4193
Date:                Tue, Mar 30 2021   R-squared (Overall):             -0.6214
Time:                        08:55:23   Log-likelihood                   -185.20
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      36.830
Entities:                          24   P-value                           0.0000
Avg Obs:                       5.3333   Distribution:                   F(2,102)
Min Obs:                       2.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             40.866
                            

### With both state and time fixed effects

In [81]:
panel_ols_stfe = PanelOLS(panel_data_l1_ols.NEET, panel_data_l1_ols[['const', 'Exp_LMP', 'STR']], entity_effects=True, time_effects=True).fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

In [82]:
print(panel_ols_stfe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   NEET   R-squared:                        0.1816
Estimator:                   PanelOLS   R-squared (Between):             -0.1802
No. Observations:                 128   R-squared (Within):               0.3225
Date:                Tue, Mar 30 2021   R-squared (Overall):             -0.1037
Time:                        10:20:05   Log-likelihood                   -154.25
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      10.764
Entities:                          24   P-value                           0.0001
Avg Obs:                       5.3333   Distribution:                    F(2,97)
Min Obs:                       2.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             10.622
                            

## Panel OLS with economic features

In [22]:
added_features = df_eco_features[df_eco_features.Time.isin(years)].reset_index(drop=True)
df_eco_neet = pd.DataFrame(columns=['Country', 'Time', 'NEET', 'Exp_LMP', 'STR', 'Min_Wage', 'LogGDP', 'CPI', 'LogDEBT'])

In [23]:
for obs in panel_data_l1.itertuples():
    country, time = obs[1], obs[2]
    new_el = added_features.loc[(added_features.Country==country)&(added_features.Time==time)]
    if len(new_el.index) > 0:
        line = {'Country':country, 'Time':time, 'NEET':obs[3], 'Exp_LMP':obs[4], 'STR':obs[5], 'Min_Wage':obs[6], 'LogGDP':new_el.LogGDP.values[0], 'CPI':new_el.CPI.values[0], 'LogDEBT':new_el.LogDEBT.values[0]}
        df_eco_neet = df_eco_neet.append(line, ignore_index=True)

In [24]:
df_eco_neet_ols = df_eco_neet.set_index(['Country', 'Time'])

In [25]:
df_eco_neet_ols = sm.add_constant(df_eco_neet_ols)

In [26]:
df_eco_neet_ols

const       NEET  Exp_LMP     STR      Min_Wage     LogGDP  \
Country Time                                                               
AUS     2013    1.0  13.015899     0.87  15.615  23283.766881  11.129468   
        2014    1.0  12.647472     0.93  15.612  23356.492667  11.043094   
        2015    1.0  11.831610     0.91  15.433  23641.395398  10.946512   
        2016    1.0  11.352150     0.86  15.168  23915.419580  10.819201   
        2017    1.0  10.946128     0.85  15.124  24128.731046  10.897257   
...             ...        ...      ...     ...           ...        ...   
USA     2014    1.0  15.047262     0.28  15.435  16285.276127  10.916265   
        2015    1.0  14.380193     0.28  15.354  16265.980260  10.947985   
        2016    1.0  14.118049     0.26  15.216  16063.328012  10.967363   
        2017    1.0  13.280724     0.24  15.182  15728.297963  11.003136   
        2018    1.0  12.685773     0.25  15.224  15353.281279  11.050834   

                   CPI   LogDEBT  
Country Time                      
AUS     2013  2.449889  4.020154  
        2014  2.487923  4.116895  
        2015  1.508367  4.161692  
        2016  1.276991  4.225249  
        2017  1.948647  4.183646  
...                ...       ...  
USA     2014  1.622223  4.909594  
        2015  0.118627  4.915812  
        2016  1.261583  4.928058  
        2017  2.130110  4.902859  
        2018  2.442583  4.913975  

[122 rows x 8 columns]

In [83]:
panel_eco_ols = PanelOLS(df_eco_neet_ols.NEET, df_eco_neet_ols.drop(['NEET', 'Min_Wage'], axis=1), time_effects=True, entity_effects=True).fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

In [84]:
print(panel_eco_ols)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   NEET   R-squared:                        0.3402
Estimator:                   PanelOLS   R-squared (Between):              0.3160
No. Observations:                 122   R-squared (Within):               0.4292
Date:                Tue, Mar 30 2021   R-squared (Overall):              0.3326
Time:                        10:29:01   Log-likelihood                   -135.81
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.1761
Entities:                          23   P-value                           0.0000
Avg Obs:                       5.3043   Distribution:                    F(5,89)
Min Obs:                       2.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             7.3963
                            

## Panel OLS with economic and education features

Ici, supprimer juste `Min_Wage` c'est pas suffisant pour retirer la multicollinéarité, il faut enlever en plus `Exp_educ`. Après c'est une variable de contrôle alors je sais pas si c'est si grave que ça.

In [29]:
reduced_obs = df_eco_neet[df_eco_neet.Time.isin(years[:-1])].reset_index(drop=True)
df_eco_educ_neet = pd.DataFrame(columns=['Country', 'Time', 'NEET', 'Exp_LMP', 'STR', 'Min_Wage', 'LogGDP', 'CPI', 'LogDEBT', 'Years_schooling', 'Avg_class_size', 'Exp_educ'])

In [30]:
for obs in reduced_obs.itertuples():
    country, time = obs[1], obs[2]
    new_el = df_educ_features_l1.loc[(df_educ_features_l1.Country==country)&(df_educ_features_l1.Time==time)]
    if len(new_el.index) > 0:
        line = {'Country':country, 'Time':time, 'NEET':obs[3], 'Exp_LMP':obs[4], 'STR':obs[5], 'Min_Wage':obs[6], 'LogGDP':obs[7], 'CPI':obs[8], 'LogDEBT':obs[9], 'Years_schooling':new_el.Years_schooling.values[0], 'Avg_class_size':new_el.Avg_class_size.values[0], 'Exp_educ':new_el.Exp_educ.values[0]}
        df_eco_educ_neet = df_eco_educ_neet.append(line, ignore_index=True)

In [31]:
df_eco_educ_neet

,Country,Time,NEET,Exp_LMP,STR,Min_Wage,LogGDP,CPI,LogDEBT,Years_schooling,Avg_class_size,Exp_educ
0,AUS,2013,13.015899,0.87,15.615,23283.766881,11.129468,2.449889,4.020154,12.6,23.725,9241.9922
1,AUS,2014,12.647472,0.93,15.612,23356.492667,11.043094,2.487923,4.116895,12.7,23.859,9257.9980
2,AUS,2015,11.831610,0.91,15.433,23641.395398,10.946512,1.508367,4.161692,12.8,23.821,9524.7178
3,AUS,2016,11.352150,0.86,15.168,23915.419580,10.819201,1.276991,4.225249,12.9,23.669,10022.5670
4,AUS,2017,10.946128,0.85,15.124,24128.731046,10.897257,1.948647,4.183646,12.9,23.613,10238.4130
...,...,...,...,...,...,...,...,...,...,...,...,...
74,ESP,2013,27.151373,3.53,13.756,13277.313554,10.277102,1.408546,4.668683,9.5,21.606,6967.5908
75,ESP,2014,24.314959,3.11,13.544,13297.375346,10.290841,-0.150870,4.783094,9.7,21.713,7052.4565
76,ESP,2015,22.820313,2.58,13.655,13432.601777,10.155491,-0.500461,4.762817,9.7,21.878,7356.7319
77,ESP,2016,21.675491,2.30,13.564,13594.355451,10.185102,-0.202672,4.764995,9.8,21.875,7805.0806


In [32]:
df_eco_educ_neet_ols = df_eco_educ_neet.set_index(['Country', 'Time'])

In [33]:
df_eco_educ_neet_ols = sm.add_constant(df_eco_educ_neet_ols)

In [85]:
panel_eco_educ_ols = PanelOLS(df_eco_educ_neet_ols.NEET, df_eco_educ_neet_ols.drop(['NEET', 'Min_Wage'], axis=1), time_effects=True, entity_effects=True).fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

In [86]:
print(panel_eco_educ_ols)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   NEET   R-squared:                        0.3810
Estimator:                   PanelOLS   R-squared (Between):             -1.1318
No. Observations:                  79   R-squared (Within):               0.1615
Date:                Tue, Mar 30 2021   R-squared (Overall):             -0.9327
Time:                        10:42:43   Log-likelihood                   -88.129
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.8463
Entities:                          17   P-value                           0.0014
Avg Obs:                       4.6471   Distribution:                    F(8,50)
Min Obs:                       3.0000                                           
Max Obs:                       5.0000   F-statistic (robust):            -12.415
                            

to print summary to latex format: print(panel_eco_educ_ols.summary.as_latex())

## Panel OLS with economic, education and labour market features

In [37]:
df_labour_features.columns

Index(['Country', 'Time', 'protection_of_workers', 'short_time_workers',
       'involuntary_pt_workers', 'ft_and_pt_employ',
       'marginally_attached_workers', 'employees_bargain', 'years_schooling',
       'avg_class_size', 'educ_spendings'],
      dtype='object')

In [73]:
reduced_obs = df_eco_neet[df_eco_neet.Time.isin(years[:-1])].reset_index(drop=True)
df_eco_educ_lm_neet = pd.DataFrame(columns=['Country', 'Time', 'NEET', 'Exp_LMP', 'STR', 'Min_Wage', 'LogGDP', 'CPI', 'LogDEBT', 'Years_schooling', 'Avg_class_size', 'Exp_educ'])

In [74]:
for obs in reduced_obs.itertuples():
    country, time = obs[1], obs[2]
    new_el = df_labour_features.loc[(df_labour_features.Country==country)&(df_labour_features.Time==time)]
    if len(new_el.index) > 0:
        line = {'Country':country, 'Time':time, 'NEET':obs[3], 'Exp_LMP':obs[4], 'STR':obs[5], 'Min_Wage':obs[6], 'LogGDP':obs[7], 'CPI':obs[8], 'LogDEBT':obs[9], 'Years_schooling':new_el.years_schooling.values[0], 'Avg_class_size':new_el.avg_class_size.values[0], 'Exp_educ':new_el.educ_spendings.values[0]}
        df_eco_educ_lm_neet = df_eco_educ_lm_neet.append(line, ignore_index=True)

In [75]:
df_eco_educ_lm_neet

,Country,Time,NEET,Exp_LMP,STR,Min_Wage,LogGDP,CPI,LogDEBT,Years_schooling,Avg_class_size,Exp_educ
0,AUS,2014,12.647472,0.93,15.612,23356.492667,11.043094,2.487923,4.116895,12.7,23.785,8107.4346
1,AUS,2016,11.352150,0.86,15.168,23915.419580,10.819201,1.276991,4.225249,12.9,22.158,8795.3633
2,EST,2015,12.848400,0.63,13.347,8264.357819,9.771226,-0.492326,2.536382,12.7,18.148,5838.8569
3,FRA,2014,16.251743,3.02,19.641,21745.708496,10.669217,0.507759,4.788783,11.4,25.262,6860.5566
4,DEU,2015,8.567577,1.52,15.447,23382.762030,10.623440,0.514421,4.382591,14.1,24.056,8419.0049
5,HUN,2013,20.482632,1.21,10.631,8268.343744,9.524239,1.733200,4.578753,12.0,20.863,5169.8901
6,HUN,2014,17.478123,1.21,11.460,8583.174144,9.565705,-0.227566,4.613330,11.8,20.851,3480.1599
7,LVA,2013,15.771196,0.55,11.162,6342.214871,9.623825,-0.029455,3.818739,12.7,14.420,5851.5640
8,LVA,2014,14.520842,0.55,11.091,7101.521415,9.662278,0.620492,3.929740,12.9,14.779,6474.9941
9,LVA,2015,12.961889,0.55,11.133,7951.462567,9.530582,0.174241,3.832175,12.8,15.038,6594.3418


In [76]:
df_eco_educ_lm_neet_ols = df_eco_educ_lm_neet.set_index(['Country', 'Time'])

In [77]:
df_eco_educ_lm_neet_ols = sm.add_constant(df_eco_educ_lm_neet_ols)

In [78]:
panel_eco_educ_lm_ols = PanelOLS(df_eco_educ_lm_neet_ols.NEET, df_eco_educ_lm_neet_ols.drop(['NEET', 'Avg_class_size'], axis=1), time_effects=True, entity_effects=True).fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

ZeroDivisionError: division by zero